In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

pd.options.display.max_rows = 47276
pd.options.display.max_columns = 20000

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder

In [2]:
root = os.path.join(os.getcwd(), 'DATA')

In [3]:
df_raw = pd.read_excel(os.path.join(root, 'train.xlsx'))
cpi = pd.read_csv(os.path.join(root, 'preprocessed_cpi.csv'))
oil = pd.read_csv(os.path.join(root, 'preprocessed_oil.csv'))
weather = pd.read_csv(os.path.join(root, 'preprocessed_weather.csv'))
kor_weather = pd.read_csv(os.path.join(root, 'preprocessed_weather_korea.csv'))
exchange = pd.read_csv(os.path.join(root, 'preprocessed_exchange.csv'))

In [4]:
from datetime import date, timedelta


def set_week(df, date):
    '''
    df : datetime 형식의 컬럼을 가지고 있는 dataframe
    date : df에서 datetime 형식을 가진 컬럼명
    return : date의 연도 컬럼과 주차 컬럼을 추가한 dataframe
    '''
    df[date] = pd.to_datetime(df[date])
    df[date] = df[date].dt.date
    df['year'] = df.apply(func=lambda x: x[date].isocalendar()[0], axis=1)
    df['week'] = df.apply(func=lambda x: x[date].isocalendar()[1], axis=1)

################################################################################################################################################################

def fillna(data, all_nation, major_nation , method=None):
  
  '''

  주요 수출구 결측치 제거 함수
  all_nation = 모든 수출국
  major_nation = 주요 수출국 ['칠레', '중국', '노르웨이', '페루', '태국', '베트남']
  method = ffill, bfill

  '''

  for nation in all_nation:
    for m_nation in major_nation:
      if nation == m_nation:

        if method == 'ffill':
          data[data['CTRY_2'] == m_nation].fillna(method='ffill', inplace=True)

        elif method == 'bfill':
          data[data['CTRY_2'] == m_nation].fillna(method='ffill', inplace=True)

################################################################################################################################################################

def na_list(data):

  '''
  결측치 컬럼 확인
  '''
  lst = []
  for col in data.columns:
    if data[col].isna().sum() > 0 :
      lst.append(col)
    else: pass

  return lst

################################################################################################################################################################

def drop_col(data):
  '''
  결측치 컬럼 제거
  '''
  lst = na_list(data)
  data.drop(lst, axis=1,inplace=True)

################################################################################################################################################################

def scaler_col(data, col, method):
  '''
  전처리 함수
  data 
  col : ['cpi_total', 'cpi_fish', 'oil', 'wind_kr', 'temperature_kr','water_temp_kr']
  method : standard, minmax, robust, log
  '''
  if method == 'standard':
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[col])

  elif method == 'minmax':
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[col])

  elif method == 'robust':
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[col])

  elif method == 'log':
    data[col] = np.log1p(data[col])

################################################################################################################################################################

def make_quarter(data, col):
  '''
  분기 행 추가
  col : 'REG_DATE'
  '''
  data[col] = pd.to_datetime(data[col])
  data['Quarter'] = data[col].dt.quarter
  data['Quarter_ver2'] = data['Quarter'] + (data['year'] - data['year'].min()) * 4
  dummies = pd.get_dummies(df_train['Quarter'], prefix='Quater_Dummy', drop_first=True)

  if 'Quarter' in data.columns:
    data = pd.concat([data, dummies], axis=1)
    del data['Quarter']

################################################################################################################################################################


def nation_week_df(data,days, nation, kind):
  '''
  
  국가 종류별 주차별 가격 평균 df  및 결측값 채움.
  data = df
  days = set(df_copy['REG_DATE'])
  nation = CTRY_2_국가이름
  kind = P_NAME_종류이름
  
  '''
  kind_lst = []
  price_lst = []
  temp_df = pd.DataFrame()

  for day in days:
    nation_kind = data[(data[nation] == 1) & (data['REG_DATE'] == day) & (data[kind] == 1)]
    kind_lst.append(nation_kind)
  kind_df = pd.concat(kind_lst, axis=0).sort_values(by='REG_DATE').reset_index(drop=True)

  for day in days:
    price_lst.append(kind_df[ (kind_df['REG_DATE'] == day) ]['P_PRICE'].mean())

  temp_df = temp_df.append(price_lst)
  temp_df = temp_df.dropna().reset_index(drop=True)
  temp = list(temp_df.iloc[:,0])
  kind_df = kind_df.drop_duplicates('REG_DATE').reset_index(drop=True)
  
  kind_df['P_PRICE'] = temp
  kind_df['datetime'] = kind_df['REG_DATE']
  kind_df.set_index('REG_DATE', inplace=True)
  kind_df = kind_df.asfreq('W-mon')

  kind_df.fillna(method = 'ffill', inplace= True)

  return kind_df

In [5]:
set_week(df_raw, 'REG_DATE')

In [6]:
df_train = pd.merge(df_raw, cpi, how='left', on=['year','week'])
df_train = pd.merge(df_train, oil, how='left', on=['year','week'])
df_train = pd.merge(df_train, weather, how='left', on=['year','week', 'CTRY_1'])
df_train = pd.merge(df_train, kor_weather, how='left', on=['year','week'])
df_train = pd.merge(df_train, exchange, how='left', on=['year','week','CTRY_2'])
df_train.drop('P_TYPE', axis=1, inplace=True)

In [7]:
all_nation = list(set(df_train['CTRY_2']))
major_nation = ['칠레', '중국', '노르웨이', '페루', '태국', '베트남']

In [8]:
fillna(data = df_train, all_nation = all_nation, major_nation = major_nation, method = 'ffill')

In [9]:
drop_col(df_train) ## 결측 행 제거

In [10]:
df_train = df_train.drop_duplicates() # 왜인지 모르겠지만 중복행이 존재함

In [11]:
scaler_lst = ['cpi_total', 'cpi_fish', 'oil', 'wind_kr', 'temperature_kr','water_temp_kr']

In [12]:
scaler_col(df_train, scaler_lst, method='log')

In [13]:
make_quarter(df_train, 'REG_DATE')

In [14]:
import_type_list = set()
for tmp in df_train.P_IMPORT_TYPE.unique():
    for a in tmp.split(','):
        import_type_list.add(a)

for name in import_type_list:
    df_train[name] = 0
    df_train.loc[df_train['P_IMPORT_TYPE'].str.contains(name, regex=False), name] = 1

In [15]:
df_train.drop('P_IMPORT_TYPE', axis=1, inplace=True)

In [16]:
dummy = ['CTRY_1','CTRY_2','P_PURPOSE','CATEGORY_1','CATEGORY_2','P_NAME']
df_train = pd.get_dummies(data=df_train, columns = dummy)

#############################

In [17]:
day_list = set(df_train['REG_DATE'])

In [18]:
kind_df = nation_week_df(data = df_train, days =  day_list, nation = 'CTRY_2_중국', kind = 'P_NAME_흰다리새우')

In [20]:
kind_df.index

DatetimeIndex(['2015-12-28', '2016-01-04', '2016-01-11', '2016-01-18',
               '2016-01-25', '2016-02-01', '2016-02-08', '2016-02-15',
               '2016-02-22', '2016-02-29',
               ...
               '2019-10-28', '2019-11-04', '2019-11-11', '2019-11-18',
               '2019-11-25', '2019-12-02', '2019-12-09', '2019-12-16',
               '2019-12-23', '2019-12-30'],
              dtype='datetime64[ns]', name='REG_DATE', length=210, freq='W-MON')

In [19]:
kind_df.head(50)

,P_PRICE,year,week,cpi_total,cpi_fish,oil,wind_kr,temperature_kr,water_temp_kr,Quarter,Quarter_ver2,간,냉장,염장,눈살,줄기,포장횟감,건조,창난,지느러미,내장,머리살,머리_외화획득용,훈제,필렛(F),한쪽껍질붙은,횟감,동체,턱살,다리,볼살,곤이,머리,슬라이스(S),절단,캐비아대용,활,알,개아지살,난포선,껍질,꼬리_외화획득용,살,집게다리,자숙,목살,외투막,냉동,CTRY_1_가나,CTRY_1_감비아,CTRY_1_그리스,CTRY_1_그린란드,CTRY_1_기니,CTRY_1_기니비사우,CTRY_1_나미비아,CTRY_1_남아프리카 공화국,CTRY_1_네덜란드,CTRY_1_노르웨이,CTRY_1_뉴질랜드,CTRY_1_니카라과,CTRY_1_대만,CTRY_1_대한민국,CTRY_1_라이베리아,CTRY_1_라트비아,CTRY_1_러시아,CTRY_1_루마니아,CTRY_1_리비아,CTRY_1_마다가스카르,CTRY_1_말레이시아,CTRY_1_멕시코,CTRY_1_모로코,CTRY_1_모리타니,CTRY_1_모잠비크,CTRY_1_몰타,CTRY_1_미국,CTRY_1_미얀마,CTRY_1_미크로네시아 연방,CTRY_1_바누아투,CTRY_1_바레인,CTRY_1_방글라데시,CTRY_1_베네수엘라,CTRY_1_베트남,CTRY_1_불가리아,CTRY_1_브라질,CTRY_1_사우디아라비아,CTRY_1_사이프러스,CTRY_1_세네갈,CTRY_1_세이셸,CTRY_1_세인트빈센트 그레나딘,CTRY_1_소말리아,CTRY_1_수리남,CTRY_1_스리랑카,CTRY_1_스페인,CTRY_1_시에라리온,CTRY_1_싱가포르,CTRY_1_아랍에미리트,CTRY_1_아르헨티나,CTRY_1_아이슬란드,CTRY_1_아일랜드,CTRY_1_알제리,CTRY_1_앙골라,CTRY_1_에스토니아,CTRY_1_에콰도르,CTRY_1_영국,CTRY_1_오만,CTRY_1_우루과이,CTRY_1_우크라이나,CTRY_1_이란,CTRY_1_이집트,CTRY_1_이탈리아,CTRY_1_인도,CTRY_1_인도네시아,CTRY_1_일본,CTRY_1_중국,CTRY_1_칠레,CTRY_1_캐나다,CTRY_1_콜롬비아,CTRY_1_쿠바,CTRY_1_쿡 제도,CTRY_1_크로아티아,CTRY_1_키리바시,CTRY_1_태국,CTRY_1_터키,CTRY_1_튀니지,CTRY_1_파나마,CTRY_1_파키스탄,CTRY_1_파푸아뉴기니,CTRY_1_팔라우,CTRY_1_페루,CTRY_1_포르투갈,CTRY_1_포클랜드 제도,CTRY_1_프랑스,CTRY_1_피지,CTRY_1_필리핀,CTRY_1_호주,CTRY_2_가나,CTRY_2_감비아,CTRY_2_그리스,CTRY_2_기니,CTRY_2_기니비사우,CTRY_2_기타(ZZ),CTRY_2_나미비아,CTRY_2_남아프리카 공화국,CTRY_2_네덜란드,CTRY_2_노르웨이,CTRY_2_뉴질랜드,CTRY_2_대만,CTRY_2_덴마크,CTRY_2_독일,CTRY_2_라이베리아,CTRY_2_러시아,CTRY_2_루마니아,CTRY_2_마다가스카르,CTRY_2_말레이시아,CTRY_2_멕시코,CTRY_2_모로코,CTRY_2_모리셔스,CTRY_2_모리타니,CTRY_2_모잠비크,CTRY_2_몰타,CTRY_2_미국,CTRY_2_미얀마,CTRY_2_미크로네시아 연방,CTRY_2_바누아투,CTRY_2_바레인,CTRY_2_방글라데시,CTRY_2_베네수엘라,CTRY_2_베트남,CTRY_2_벨리즈,CTRY_2_북한,CTRY_2_불가리아,CTRY_2_브라질,CTRY_2_사모아,CTRY_2_사우디아라비아,CTRY_2_세네갈,CTRY_2_세이셸,CTRY_2_솔로몬 제도,CTRY_2_수리남,CTRY_2_스리랑카,CTRY_2_스웨덴,CTRY_2_스페인,CTRY_2_시에라리온,CTRY_2_싱가포르,CTRY_2_아랍에미리트,CTRY_2_아르헨티나,CTRY_2_아이슬란드,CTRY_2_아일랜드,CTRY_2_앙골라,CTRY_2_에콰도르,CTRY_2_영국,CTRY_2_오만,CTRY_2_우루과이,CTRY_2_우크라이나,CTRY_2_이란,CTRY_2_이집트,CTRY_2_이탈리아,CTRY_2_인도,CTRY_2_인도네시아,CTRY_2_일본,CTRY_2_중국,CTRY_2_지부티,CTRY_2_칠레,CTRY_2_캐나다,CTRY_2_콜롬비아,CTRY_2_쿠바,CTRY_2_크로아티아,CTRY_2_키리바시,CTRY_2_태국,CTRY_2_터키,CTRY_2_투발루,CTRY_2_튀니지,CTRY_2_파나마,CTRY_2_파키스탄,CTRY_2_파푸아뉴기니,CTRY_2_팔라우,CTRY_2_페루,CTRY_2_포르투갈,CTRY_2_포클랜드 제도,CTRY_2_프랑스,CTRY_2_피지,CTRY_2_필리핀,CTRY_2_호주,CTRY_2_홍콩,P_PURPOSE_반송품(기타),P_PURPOSE_외화획득용 원료,P_PURPOSE_자사제품제조용,P_PURPOSE_판매용,P_PURPOSE_합작,CATEGORY_1_갑각류,CATEGORY_1_기타 수입식품,CATEGORY_1_알 곤이류,CATEGORY_1_어류,CATEGORY_1_연체류 해물모듬,CATEGORY_1_젓갈류 해조류 해파리,CATEGORY_1_패류 멍게류,CATEGORY_2_가리비,CATEGORY_2_가물치,CATEGORY_2_가사리,CATEGORY_2_가오리,CATEGORY_2_가자미,CATEGORY_2_가재 랍스타,CATEGORY_2_갈치,CATEGORY_2_갑오징어,CATEGORY_2_개복치,CATEGORY_2_개불,CATEGORY_2_게,CATEGORY_2_고동,CATEGORY_2_고등어,CATEGORY_2_고시래기,CATEGORY_2_골뱅이,CATEGORY_2_광어 넙치,CATEGORY_2_김,CATEGORY_2_꼬막,CATEGORY_2_꼴뚜기,CATEGORY_2_꽁치 학꽁치,CATEGORY_2_꽁치 확꽁치,CATEGORY_2_낙지,CATEGORY_2_날치알,CATEGORY_2_남극빙어,CATEGORY_2_노래미,CATEGORY_2_농어,CATEGORY_2_능성어 붉바리 바리,CATEGORY_2_다시마,CATEGORY_2_달고기,CATEGORY_2_대구,CATEGORY_2_대구알,CATEGORY_2_도다리,CATEGORY_2_도미 감성돔 돔류,CATEGORY_2_망둑어,CATEGORY_2_멍게,CATEGORY_2_메기 동자개,CATEGORY_2_메로,CATEGORY_2_멸치,CATEGORY_2_명란(명태알),CATEGORY_2_명태,CATEGORY_2_문어,CATEGORY_2_물메기(곰치),CATEGORY_2_미꾸라지,CATEGORY_2_미역,CATEGORY_2_민물붕어,CATEGORY_2_민어 점성어,CATEGORY_2_밀크피시,CATEGORY_2_바지락,CATEGORY_2_방어,CATEGORY_2_밴댕이,CATEGORY_2_버터플라이 킹피쉬,CATEGORY_2_벤자리 알롱이,CATEGORY_2_병어,CATEGORY_2_보리멸,CATEGORY_2_복어,CATEGORY_2_부세,CATEGORY_2_붉평치(만다이 꽃돔),CATEGORY_2_삼치,CATEGORY_2_상어 고래,CATEGORY_2_새우,CATEGORY_2_샛돔류알,CATEGORY_2_서대 박대 페루다,CATEGORY_2_성게알,CATEGORY_2_소라,CATEGORY_2_송어,CATEGORY_2_쏘가리,CATEGORY_2_아귀,CATEGORY_2_양미리 정어리,CATEGORY_2_양태,CATEGORY_2_어류 기타,CATEGORY_2_연어,CATEGORY_2_연어알,CATEGORY_2_열빙어(시샤모),CATEGORY_2_열빙어(시샤모)알,CATEGORY_2_오징어,CATEGORY_2_옥돔,CATEGORY_2_우럭 볼락,CATEGORY_2_우렁 다슬기,CATEGORY_2_은민대구알,CATEGORY_2_임연수,CATEGORY_2_잉어,CATEGORY_2_자라,CATEGORY_2_장어,CATEGORY_2_재첩,CATEGORY_2_적어 눈볼대,CATEGORY_2_전갱기 매가리,CATEGORY_2_전갱이 매가리,CATEGORY_2_전복,CATEGORY_2_전어,CATEGORY_2_조개,CATEGORY_2_조개 백합 대합,CATEGORY_2_조기 보구치 강다리,CATEGORY_2_조기 보구치 강다리,CATEGORY_2_준